In [1]:
import pandas as pd
import numpy as np
import os, yaml, wandb, pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from rl import make_transition, make_transition_for_AKI, imvt

from tqdm import tqdm

if os.getcwd()[-4:] == "code":
    os.chdir('../')
    
with open(os.path.join("./code/params.yaml")) as f:
        params = yaml.safe_load(f)
    
np.random.seed(params['random_seed'])
torch.manual_seed(params['random_seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(params['random_seed'])
    torch.cuda.manual_seed_all(params['random_seed'])

In [2]:
wandb.init(
    name="AKI_smooth_l1_non_aki_label",
    project="AIAKI_IMV_LSTM",
    config={
        "learning_rate":params["learning_rate"],
        "epoch":params['epoch'],
        "batch_size":params["minibatch_size"],
        "n_units":params["n_units"],
        "update_freq":params["update_freq"]
    }
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chanreverse (dahs). Use `wandb login --relogin` to force relogin


In [11]:
train_loader, train_len = make_transition_for_AKI('./code/train_non_aki.parquet',rolling_size=24,batch_size=64,istrain=True)
val_loader, val_len = make_transition_for_AKI('./code/val_non_aki.parquet',rolling_size=24,batch_size=256,istrain=False)

100%|██████████| 10664/10664 [01:11<00:00, 149.14it/s]


Finished train data transition for AKI


100%|██████████| 1523/1523 [00:03<00:00, 503.44it/s]


Finished val or test data trnsition for AKI


In [18]:
device = 'cuda:0'
network = imvt(input_dim=122, output_dim=params['num_actions'], n_units=params['n_units'], device=device).to(device)
target_network = imvt(input_dim=122, output_dim=params['num_actions'], n_units=params['n_units'], device=device).to(device)

In [19]:
Model="AKI_smooth_l1_non_aki_label"

epoch = params['epoch']
gamma = params['gamma']
optimizer = optim.Adam(network.parameters(), lr=params['learning_rate'], amsgrad=True)

update_freq = params['update_freq']
train_loss_history = []
val_loss_history = []

for i in range(epoch):
    loss_train = 0
    update_counter = 0
    for s,a,r,s2,t in tqdm(train_loader):
        s = s.to(device)
        a = a.to(device)
        r = r.to(device)
        s2 = s2.to(device)
        t = t.to(device)

        q,_,_ = network(s)
        q_pred = q.gather(1, a).squeeze()
        
        with torch.no_grad():
            q2,_,_ = target_network(s2)
            q2_net, _, _ = network(s2)

        q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze(1).detach()

        bellman_target = torch.clamp(r, max=0.0, min=-1.0) + gamma * torch.clamp(q2_max.detach(), max=0.0, min=-1.0)*(1-t)
        loss = F.smooth_l1_loss(q_pred, bellman_target)

        optimizer.zero_grad()
        loss.backward()
        loss_train += loss.item()
        optimizer.step()

        update_counter += 1
        if update_counter == update_freq:
            target_network.load_state_dict(network.state_dict())
            update_counter = 0

    with torch.no_grad():
        loss_val = 0

        for s,a,r,s2,t in val_loader:
            s = s.to(device)
            a = a.to(device)
            r = r.to(device)
            s2 = s2.to(device)
            t = t.to(device)

            q,_,_ = network(s)
            q2,_,_ = target_network(s2)
            q2 = q2.detach()
            q_pred = q.gather(1, a).squeeze()

            q2_net,_,_ = network(s2)
            q2_net = q2_net.detach()
            q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze()

            bellman_target = torch.clamp(r, max=0.0, min=-1.0) + gamma * torch.clamp(q2_max.detach(), max=0.0, min=-1.0)*(1-t)
            loss = F.smooth_l1_loss(q_pred, bellman_target)
            loss_val += loss.item()
            
    wandb.log({"Iter:": i, "train:":loss_train/train_len, "val:":loss_val/val_len})
    print("Iter:", i, "train:", loss_train/train_len, "val:",loss_val/val_len)
    if not os.path.exists('checkpoint_%s'%Model):
        os.makedirs('./checkpoint_%s'%Model)
    torch.save(network.state_dict(), './checkpoint_%s/checkpoint_%i.pt'%(Model,i))
    torch.save(optimizer.state_dict(), './checkpoint_%s/optim.pth'%Model)
    train_loss_history.append(loss_train/train_len)
    val_loss_history.append(loss_val/val_len)
    
    with open('./checkpoint_%s/train_loss_%s'%(Model,Model), 'wb') as f:
        pickle.dump(train_loss_history, f)
        
    with open('./checkpoint_%s/val_loss_%s'%(Model,Model), 'wb') as f:
        pickle.dump(val_loss_history, f)

13620it [10:17, 22.07it/s]                           


Iter: 0 train: 4.266620782946649e-09 val: 2.0416814742910297e-06


13620it [10:03, 22.56it/s]                           


Iter: 1 train: 2.979897773429268e-12 val: 2.0417066236533477e-06


13620it [10:18, 22.03it/s]                           


...

Iter: 91 train: 5.5416248826401085e-14 val: 2.04177906650694e-06


13620it [11:10, 20.30it/s]                           


Iter: 92 train: 5.2634970566990665e-14 val: 2.0417796867420473e-06


  4%|▍         | 546/13194 [00:25<09:41, 21.76it/s]


KeyboardInterrupt: 